In [1134]:
# ==============================
# Standard Library Imports
# ==============================
from enum import Enum
import random

# ==============================
# Third-party Library Imports
# ==============================
import matplotlib.pyplot as plt
from IPython.display import display, Latex
from matplotlib.ticker import MultipleLocator
import numpy as np  # Original numpy
import pennylane as qml
import scipy as sp

import torch

# Pennylane numpy
from pennylane import numpy as pnp 

In [1135]:
# ==============================
# Setup for Quantum Computations
# ==============================

# PennyLane settings
dev = qml.device('default.mixed', wires=1)

# Define Hamiltonian for quantum computations
coeffs = [-0.5]
obs = [qml.PauliZ(0)]
hamiltonian = qml.Hamiltonian(coeffs, obs)

In [1136]:
Tau_global = 5e-2   # Dephase tau
Paras_global = torch.zeros(2)
Phi_global = 0  
Gamma_ps = 0

In [1137]:
# Phi_global

$$
Let, e^{-t/T_2} = e^{-\tau}
$$

$$
\frac{1}{2} 

\begin{bmatrix}

1 & e^{(i\phi - \tau)} \\
e^{(-i\phi - \tau)} & 1

\end{bmatrix}

=

\frac{1}{2} 

\begin{bmatrix}

1 & e^{i\phi} \sqrt{1 - \gamma} \\
e^{-i\phi} \sqrt{1 - \gamma} & 1

\end{bmatrix}$$

$$

Then,
\quad \gamma = 
1 - e^{-2 \tau}
$$

$$ 
e^{-\tau} = \sqrt{1 - \gamma}
$$

In [1138]:
@qml.qnode(dev, interface='torch', diff_method='backprop')
def circuit(paras):
    global Phi_global
    phi = Phi_global
    
    # Set dephase factor 
    theta_x, phi_z = paras

    qml.RX(pnp.pi/2, wires = 0)
    # qml.Hadamard(wires=0)

    qml.ApproxTimeEvolution(hamiltonian, phi, 1)
    
    qml.RZ(phi_z, wires = 0)  # phi_z
    
    qml.RX(theta_x, wires = 0)  # theta_x
    
    # return qml.expval(hamiltonian)
    return qml.density_matrix(wires = 0)

In [1139]:
# paras = torch.tensor([np.pi/2, np.pi/2], dtype=torch.complex128)
# Paras_global = paras

# Phi_global = torch.tensor([np.pi*2])

# circuit(paras)

In [1140]:
# CFI = qml.qinfo.classical_fisher(circuit)(paras)

# CFI

In [1141]:
# PHI = torch.arange(0, np.pi/2, 1e-1)
# for phi_idx, phi_current in enumerate(PHI):
#     Paras_global = paras
#     cal = qml.qinfo.classical_fisher(circuit)(phi_current)
#     print(cal)

In [1142]:
# @qml.qnode(dev, interface='torch', diff_method='backprop')
# def circuit_test(paras):
#     density_matrix = circuit(Paras_global)
#     qml.QubitDensityMatrix(density_matrix, wires = 0)
    
#     return qml.density_matrix(wires=0)

In [1143]:
# circuit_test(paras)

In [1144]:
# CFI = qml.qinfo.classical_fisher(circuit)(paras)

# CFI

In [1145]:
@qml.qnode(dev, interface='torch', diff_method='backprop')
def Post_selection_Dephase(phi):
    """ Take qnode from circuit_1 and calculate decoherence using kraus operator.
    
    Args:
        phi (float): Phi for Time-approximation. Pass by global variables:'Phi_global'

    Returns:
        qml.density_matrix: Density matrix of full qnode
    """
    
    global Paras_global, Phi_global, Gamma_ps
    # Phi_global = torch.tensor([phi.item()])
    Phi_global = phi
    
    # Get density matrix from circuit_1    
    density_matrix = circuit(Paras_global)
    # density_matrix = density_matrix.clone().detach().requires_grad_(True)
    # density_matrix = torch.tensor(circuit(Paras_global), dtype=torch.complex128)
    qml.QubitDensityMatrix(density_matrix, wires = 0)
    
    # Kraus operator for 2*2 matrix
    K = torch.tensor([
        [pnp.sqrt(1 - Gamma_ps), 0],
        [0, 1]
    ], dtype=torch.complex128)
    
    Numerator = K @ density_matrix @ K.conj().T
    Denominator = torch.trace(Numerator)
    rho_ps = Numerator / Denominator

    qml.QubitDensityMatrix(rho_ps, wires = 0)
    # return qml.expval(hamiltonian) 
    return qml.density_matrix(wires = 0) 

In [1146]:
paras = torch.tensor([np.pi/4, np.pi/4], dtype=torch.complex128)
Paras_global = paras

# phi = torch.tensor([np.pi*2])
phi = pnp.array([np.pi*2])


Post_selection_Dephase(phi)

tensor([[0.2500+0.0000j, 0.3536+0.2500j],
        [0.3536-0.2500j, 0.7500+0.0000j]], dtype=torch.complex128)

In [1147]:
Phi_global.dtype

dtype('float64')

In [1148]:
PHI = pnp.arange(0, np.pi/2, 1e-1)

PHI.dtype

dtype('float64')

In [1149]:
phi_given = pnp.array([0.5], requires_grad = True)

qml.qinfo.classical_fisher(Post_selection_Dephase)(phi_given)

TypeError: The inputs given to jacobian must be either a Tensor or a tuple of Tensors but the given inputs has type <class 'pennylane.numpy.tensor.tensor'>.

In [ ]:
# PHI = np.arange(0, np.pi/2, 1e-1)

for idx, phi_current in enumerate(PHI):
    # phi_in = pnp.array([phi_current])
    cal = qml.qinfo.classical_fisher(Post_selection_Dephase)(phi_current)
    
    print(cal)

TypeError: The inputs given to jacobian must be either a Tensor or a tuple of Tensors but the given inputs has type <class 'numpy.float64'>.

In [ ]:
CFI = qml.qinfo.classical_fisher(Post_selection_Dephase)(phi)
CFI

tensor([[0.]], dtype=torch.float64, grad_fn=<MmBackward0>)

In [ ]:
@qml.qnode(dev, interface='torch', diff_method='backprop')
def circuit_test(phi):
    global Paras_global, Phi_global, Gamma_ps
    Phi_global = phi
    
    density_matrix = circuit(Paras_global)
    density_matrix = density_matrix.clone().detach().requires_grad_(True)
    qml.QubitDensityMatrix(density_matrix, wires = 0)
    
    return qml.density_matrix(wires=0)

In [ ]:
# paras = torch.tensor([np.pi/2, np.pi/2], dtype=torch.complex128)
# Paras_global = paras    # dtype=torch.complex128s

# # float
# phi = np.pi/2
# Gamma_ps = 0.8

# circuit_test(phi)

In [ ]:
# paras = torch.tensor([np.pi/2, np.pi/2], dtype=torch.complex128)
# Paras_global = paras    # dtype=torch.complex128s

# # float
# phi = np.pi/2
# Gamma_ps = 0.8

# Post_selection_Dephase(phi)

In [ ]:
paras = torch.tensor([np.pi/2, np.pi/2], dtype=torch.complex128)
Paras_global = paras    # dtype=torch.complex128s

# float
phi = np.pi/2
Gamma_ps = 0.8
 
Post_selection_Dephase(phi)

tensor([[1.2326e-33+0.0000e+00j, 0.0000e+00+3.5108e-17j],
        [0.0000e+00-4.9651e-17j, 1.0000e+00+0.0000e+00j]],
       dtype=torch.complex128, grad_fn=<ReshapeAliasBackward0>)

In [ ]:
qml.qinfo.classical_fisher(Post_selection_Dephase)(paras)

ValueError: only one element tensors can be converted to Python scalars

In [ ]:
Phi_global = np.pi/2
paras = torch.tensor([np.pi/2, np.pi/2], dtype=torch.complex128)

paras

tensor([1.5708+0.j, 1.5708+0.j], dtype=torch.complex128)

In [ ]:
qml.qinfo.classical_fisher(circuit)(paras)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/autoray/autoray.py:1770: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/Copy.cpp:276.)
  return x.to(dtype=to_backend_dtype(dtype, like=x))


tensor([[ 3.5327e-16, -2.3551e-16],
        [-2.3551e-16,  1.5701e-16]], dtype=torch.float64,
       grad_fn=<MmBackward0>)

In [ ]:
def Cost_function(paras):
    """ Calculate Classical-Fisher-Information for qnode(=Post_selection_Dephase).
    
    Args:
        paras (Numpy array): [theta_init, tau_1, tau_2, tau_d1, tau_d2, tau_d3]

    Returns:
        _type_: CFI with minus(-) sign.
    """
    
    global Paras_global, Phi_global
    Paras_global = paras
    phi = pnp.array([Phi_global])
          
    CFI = qml.qinfo.classical_fisher(Post_selection_Dephase)(phi[0])
    
    return -CFI

In [ ]:
# paras = torch.zeros(2, dtype = torch.complex128)
# paras[:2] = np.pi/2

# paras

In [ ]:
paras = np.zeros(2)
paras[:2] = np.pi/2
Paras_global = paras

Phi_global = np.pi*2
phi = pnp.array([Phi_global], requires_grad = True)

CFI = qml.qinfo.classical_fisher(Post_selection_Dephase)(phi[0])

CFI


TypeError: The inputs given to jacobian must be either a Tensor or a tuple of Tensors but the given inputs has type <class 'pennylane.numpy.tensor.tensor'>.